# Penguins in Production

This notebook creates a [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) to build an end-to-end Machine Learning system to solve the problem of classifying penguin species.

This example uses the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data).

<img src='https://imgur.com/orZWHly.png' alt='Penguins dataset' width="900">

This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

Let's ensure we are running the latest version of the SakeMaker SDK. **Restart the Kernel** after you run the following cell.

In [2]:
!pip install -q --upgrade pip
!pip install -q --upgrade awscli boto3
!pip install -q --upgrade sagemaker==2.146.0
!pip show sagemaker

Name: sagemaker
Version: 2.146.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, jsonschema, numpy, packaging, pandas, pathos, platformdirs, protobuf, protobuf3-to-dict, PyYAML, schema, smdebug-rulesconfig
Required-by: 


In [2]:
import os
import sagemaker
import numpy as np
import boto3
import json
import pandas as pd
import numpy as np
import urllib.request
import argparse
import random
import tempfile
import tarfile
import time

from time import sleep
from datetime import datetime
from threading import Thread, Event
from pathlib import Path
from botocore.exceptions import ClientError
from sagemaker import ModelPackage
from sagemaker.deserializers import JSONDeserializer
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.inputs import (
    TrainingInput, FileSystemInput, CreateModelInput, TransformInput
)
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.tuner import HyperparameterTuner
from sagemaker.workflow.steps import (
    TuningStep, TrainingStep, ProcessingStep, CacheConfig,
    CreateModelStep, TransformStep
)
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join, JsonGet
from sagemaker.lambda_helper import Lambda
from sagemaker.model import Model
from sagemaker.model_monitor import (
    CronExpressionGenerator, EndpointInput, DefaultModelMonitor, ModelQualityMonitor, DatasetFormat
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.parameters import (
    ParameterInteger, ParameterString, ParameterFloat, ParameterBoolean
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.quality_check_step import (
    DataQualityCheckConfig, QualityCheckStep, ModelQualityCheckConfig
)
from sagemaker.workflow.model_step import ModelStep
from sagemaker.parameter import IntegerParameter
from sagemaker.predictor import Predictor
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.serializers import JSONSerializer
from sagemaker.tensorflow import TensorFlow, TensorFlowProcessor
from sagemaker.tensorflow.model import TensorFlowModel, TensorFlowPredictor
from sagemaker.transformer import Transformer
from sagemaker.sklearn.processing import SKLearnProcessor

Let's start by defining a few variables and constants we'll use throughout this notebook:

* `sagemaker_client`: We'll use a [boto3 SageMaker Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) instance to access SageMaker.
* `iam_client`: We'll use a [boto3 IAM Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html) instance to access IAM.
* `role`: This is the execution role attached to this notebook. We can use this role with any of the SageMaker services that need it to ensure they run with the appropriate permissions.
* `region`: The current region attached to our session. 
* `sagemaker_session`: The current SageMaker session.

In [53]:
iam_client = boto3.client("iam")
sagemaker_client = boto3.client("sagemaker")
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

BUCKET = "mlschool"
PENGUINS_FOLDER = Path("penguins")
CODE_FOLDER = PENGUINS_FOLDER / "code"
S3_FILEPATH = f"s3://{BUCKET}/{PENGUINS_FOLDER}"
LOCAL_FILEPATH = Path(PENGUINS_FOLDER)/ "data.csv"
MODEL_PACKAGE_GROUP_NAME = "penguins"

## Downloading the Dataset

Let's download the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data) and store it in S3.

The SageMaker Pipeline will use the data stored in this S3 location. A production application can add more data to the exact location, and the Pipeline will still work.

In [31]:
!aws s3api create-bucket --bucket $BUCKET

# Create the local folder if it doesn't exist.
PENGUINS_FOLDER.mkdir(parents=True, exist_ok=True)

# Download the official Penguins dataset and store it locally.
urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    LOCAL_FILEPATH
)

# Upload the dataset to S3. We need to do this to make it available to 
# the preprocessing step.
INPUT_DATA_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(LOCAL_FILEPATH), 
    desired_s3_uri=S3_FILEPATH,
)

print(f"Dataset S3 location: {INPUT_DATA_URI}")

Dataset S3 location: s3://mlschool/penguins/data.csv


We can now load and display the dataset.

In [6]:
df = pd.read_csv(LOCAL_FILEPATH)
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


## Remote Scripts

### Preprocessing Script

Let's create a script to do feature engineering on the original dataset. We will run this script using a SageMaker Processing Job later in this session.

The script should split the data into train, validation, and test sets so we can later train and evaluate a model. We will save the Scikit-Learn pipeline that we use to preprocess the data to use it during inference time.

The script uses the [np.split()](https://numpy.org/doc/stable/reference/generated/numpy.split.html) function to split the dataset into three sets in the following way:

1. The train set will use the top 70% of the data.
2. The validation set will use 15% of the data, starting with the sample after the 70% used for the train set.
3. Finally, the test set will use the remaining 15% of the data.

Pay special attention to the way the Scikit-Learn pipeline `preprocessor` is used to process the three sets:

* First, we use the `fit_transform()` to fit the pipeline on the train set.
* Then, we consecutively transform the validation and test sets using `transform()`.

Always use `fit_transform()` on the training data to fit the scaling parameters we need to transform the data. For example, `fit_transform()` will learn the mean and variance of the features of the training set. It can then use these same parameters to scale the validation and test sets.

That's why we want to save this Scikit-Learn pipeline to use later to scale production data using the same parameters we learned on the train set.

In [7]:
%%writefile {PENGUINS_FOLDER}/preprocessor.py

import os
import numpy as np
import pandas as pd
import tempfile

from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pickle import dump


# This is the location where the SageMaker Processing job
# will save the input dataset.
BASE_DIR = "/opt/ml/processing"
DATA_FILEPATH = Path(BASE_DIR) / "input" / "data.csv"


def save_splits(base_dir, train, validation, test):
    """
    One of the goals of this script is to output the three
    dataset splits. This function will save each of these
    splits to disk.
    """
    
    train_path = Path(base_dir) / "train" 
    validation_path = Path(base_dir) / "validation" 
    test_path = Path(base_dir) / "test"
    
    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)
    
    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)
    

def save_pipeline(base_dir, pipeline):
    """
    Saves the Scikit-Learn pipeline that we used to
    preprocess the data.
    """
    pipeline_path = Path(base_dir) / "pipeline"
    pipeline_path.mkdir(parents=True, exist_ok=True)
    dump(pipeline, open(pipeline_path / "pipeline.pkl", 'wb'))
    

def generate_baseline_dataset(split_name, base_dir, X, y):
    """
    To monitor the data and the quality of our model we need to compare the 
    production quality and results against a baseline. To create those baselines, 
    we need to use a dataset to compute statistics and constraints. That dataset
    should contain information in the same format as expected by the production
    endpoint. This function will generate a baseline dataset and save it to 
    disk so we can later use it.
    
    """
    baseline_path = Path(base_dir) / f"{split_name}-baseline" 
    baseline_path.mkdir(parents=True, exist_ok=True)

    df = X.copy()
    
    # The baseline dataset needs a column containing the groundtruth.
    df["groundtruth"] = y
    df["groundtruth"] = df["groundtruth"].values.astype(str)
    
    # We will use the baseline dataset to generate baselines
    # for monitoring data and model quality. To simplify the process, 
    # we don't want to include any NaN rows.
    df = df.dropna()

    df.to_json(baseline_path / f"{split_name}-baseline.json", orient='records', lines=True)
    
    
def preprocess(base_dir, data_filepath):
    """
    Preprocesses the supplied raw dataset and splits it into a train, validation,
    and a test set.
    """
    
    df = pd.read_csv(data_filepath)
    
    numerical_columns = [column for column in df.columns if df[column].dtype in ["int64", "float64"]]
    
    numerical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    categorical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical", numerical_preprocessor, numerical_columns),
            ("categorical", categorical_preprocessor, ["island"]),
        ]
    )
    

    X = df.drop(["sex"], axis=1)
    columns = list(X.columns)
    
    X = X.to_numpy()
    
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(.7 * len(X)), int(.85 * len(X))])
    
    X_train = pd.DataFrame(train, columns=columns)
    X_validation = pd.DataFrame(validation, columns=columns)
    X_test = pd.DataFrame(test, columns=columns)
    
    y_train = X_train.species
    y_validation = X_validation.species
    y_test = X_test.species
    
    label_encoder = LabelEncoder()
    
    y_train = label_encoder.fit_transform(y_train)
    y_validation = label_encoder.transform(y_validation)
    y_test = label_encoder.transform(y_test)
    
    X_train.drop(["species"], axis=1, inplace=True)
    X_validation.drop(["species"], axis=1, inplace=True)
    X_test.drop(["species"], axis=1, inplace=True)

    # Let's generate a dataset that we can later use to compute
    # baseline statistics and constraints about the data that we
    # used to train our model.
    generate_baseline_dataset("train", base_dir, X_train, y_train)
    
    # To generate baseline constraints about the quality of the
    # model's predictions, we will use the test set.
    generate_baseline_dataset("test", base_dir, X_test, y_test)
    
    # Transform the data using the Scikit-Learn pipeline.
    X_train = preprocessor.fit_transform(X_train)
    X_validation = preprocessor.transform(X_validation)
    X_test = preprocessor.transform(X_test)
    
    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)
    
    save_splits(base_dir, train, validation, test)
    save_pipeline(base_dir, pipeline=preprocessor)
        

if __name__ == "__main__":
    preprocess(BASE_DIR, DATA_FILEPATH)


Overwriting penguins/preprocessor.py


### Training Script

This script is responsible for training a simple neural network on the train data, validating the model, and saving it so we can later use it.

In [8]:
%%writefile {PENGUINS_FOLDER}/train.py

import os
import argparse

import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(base_directory) / "model" / "001"
    model.save(model_filepath)
    
if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to the entry point
    # as script arguments. SageMaker will also provide a list of special parameters
    # that you can capture here. Here is the full list: 
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    
    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

Overwriting penguins/train.py


### Evaluation Script

This script is responsible for loading the model we created and evaluating it on the test set. Before finishing, this script will generate an evaluation report of the model.

In [9]:
%%writefile {PENGUINS_FOLDER}/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score


MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"
OUTPUT_PATH = "/opt/ml/processing/evaluation/"


def evaluate(model_path, test_path, output_path):
    # The first step is to extract the model package provided
    # by SageMaker.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    # We can now load the model from disk.
    model = keras.models.load_model(Path(model_path) / "001")
    
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test accuracy: {accuracy}")

    # Let's add the accuracy of the model to our evaluation report.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
        },
    }
    
    # We need to save the evaluation report to the output path.
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    evaluate(
        model_path=MODEL_PATH, 
        test_path=TEST_PATH,
        output_path=OUTPUT_PATH
    )

Overwriting penguins/evaluation.py


### Inference Script

Deploying the model we trained directly to an endpoint doesn't lets us control the data that goes in and comes out of the endpoint. Fortunately, SageMaker allows us to include an `inference.py` file with the model assets from where we can control how the endpoint works. You can see more information about how this works by checking the [SageMaker TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container) documentation.

We will include the inference code as part of the model assets to control the inference process on the SageMaker endpoint. SageMaker will automatically call the `handler()` function for every request to the endpoint.

In [33]:
%%writefile $CODE_FOLDER/inference.py

import os
import json
import boto3
import requests
import numpy as np
import pandas as pd

from pickle import load
from pathlib import Path


PIPELINE_FILE = Path("/tmp") / "pipeline.pkl"

# By default, we will read the S3 location of the Scikit-Learn
# pipeline from an environment variable that we'll configure
# when registering the model.
PIPELINE_S3_LOCATION = os.environ.get("PIPELINE_S3_LOCATION", None)


s3 = boto3.resource("s3")


def handler(data, context, pipeline_s3_location=PIPELINE_S3_LOCATION):
    """
    This is the entrypoint that will be called by SageMaker when the endpoint
    receives a request. You can see more information at 
    https://github.com/aws/sagemaker-tensorflow-serving-container.
    """
    print("Handling endpoint request")
    
    download_pipeline(pipeline_s3_location)
    
    instance = _process_input(data, context)
    output = _predict(instance, context)
    return _process_output(output, context)


def download_pipeline(pipeline_s3_location):
    """
    This function will download the Scikit-Learn pipeline from S3
    if it doesn't already exist. We need the pipeline to pre-process
    the data before we run it through the model.
    """
    
    s3_parts = pipeline_s3_location.split('/', 3)
    bucket = s3_parts[2]
    key = s3_parts[3]
    
    if not PIPELINE_FILE.exists():
        s3.Bucket(bucket).download_file(f"{key}/pipeline.pkl", str(PIPELINE_FILE))


def transform(payload):
    """
    This function transforms the payload in the request using the
    Scikit-Learn pipeline that we created during the preprocessing step.
    """
    
    print("Transforming input data...")

    pipeline = load(open(PIPELINE_FILE, 'rb'))
    
    island = payload.get("island", "")
    culmen_length_mm = payload.get("culmen_length_mm", 0)
    culmen_depth_mm = payload.get("culmen_depth_mm", 0)
    flipper_length_mm = payload.get("flipper_length_mm", 0)
    body_mass_g = payload.get("body_mass_g", 0)
    
    data = pd.DataFrame(
        columns=["island", "culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"], 
        data=[[
            island, 
            culmen_length_mm, 
            culmen_depth_mm, 
            flipper_length_mm, 
            body_mass_g
        ]]
    )
    
    result = pipeline.transform(data)
    return result[0].tolist()
    

def _process_input(data, context):
    print("Processing input data...")
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we can use the
        # data directly.
        endpoint_input = data
    elif context.request_content_type in ("application/json", "application/octet-stream"):
        # When the endpoint is running, we will receive a context
        # object. We need to parse the input and turn it into 
        # JSON in that case.
        endpoint_input = json.loads(data.read().decode("utf-8"))

        if endpoint_input is None:
            raise ValueError("There was an error parsing the input request.")
    else:
        raise ValueError(f"Unsupported content type: {context.request_content_type or 'unknown'}")
        
    return transform(endpoint_input)


def _predict(instance, context):
    print("Sending input data to model to make a prediction...")
    
    model_input = json.dumps({"instances": [instance]})
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we want to return
        # a fake prediction back.
        result = {
            "predictions": [
                [0.2, 0.5, 0.3]
            ]
        }
    else:
        # When the endpoint is running, we will receive a context
        # object. In that case we need to send the instance to the
        # model to get a prediction back.
        response = requests.post(context.rest_uri, data=model_input)
        
        if response.status_code != 200:
            raise ValueError(response.content.decode('utf-8'))
            
        result = json.loads(response.content)
    
    print(f"Response: {result}")
    return result


def _process_output(output, context):
    print("Processing prediction received from the model...")
    
    response_content_type = "application/json" if context is None else context.accept_header
    
    prediction = np.argmax(output["predictions"][0])
    confidence = output["predictions"][0][prediction]
    
    print(f"Prediction: {prediction}. Confidence: {confidence}")
    
    result = json.dumps({
        "prediction": str(prediction),
        "confidence": confidence
    }), response_content_type
    
    return result

Overwriting penguins/code/inference.py


### Testing Scripts Locally

We can now load the scripts we just created and run them locally to ensure they work as expected.


In [10]:
from penguins.preprocessor import preprocess
from penguins.train import train
from penguins.evaluation import evaluate


def print_baseline(split_name):
    print()
    print(f"Baseline {split_name}:")
    with open(Path(directory) / f"{split_name}-baseline" / f"{split_name}-baseline.json") as baseline:
        lines = [next(baseline) for _ in range(5)]
        
    for l in lines:
        print(l[:-1])


with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_dir=directory, 
        data_filepath=LOCAL_FILEPATH
    )
    
    print(f"Folders: {os.listdir(directory)}")
    
    print_baseline("train")
    print_baseline("test")

    # Then, we train a model using the train and 
    # validation splits.
    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=10
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(Path(directory) / "model.tar.gz", "w:gz") as tar:
        tar.add(Path(directory) / "model" / "001", arcname="001")
        
    
    # We can now call the evaluation script.
    evaluate(
        model_path=directory, 
        test_path=Path(directory) / "test",
        output_path=Path(directory) / "evaluation",
    )

Folders: ['train-baseline', 'test-baseline', 'train', 'validation', 'test', 'pipeline']

Baseline train:
{"island":"Biscoe","culmen_length_mm":52.5,"culmen_depth_mm":15.6,"flipper_length_mm":221.0,"body_mass_g":5450.0,"groundtruth":"2"}
{"island":"Biscoe","culmen_length_mm":38.8,"culmen_depth_mm":17.2,"flipper_length_mm":180.0,"body_mass_g":3800.0,"groundtruth":"0"}
{"island":"Torgersen","culmen_length_mm":39.3,"culmen_depth_mm":20.6,"flipper_length_mm":190.0,"body_mass_g":3650.0,"groundtruth":"0"}
{"island":"Biscoe","culmen_length_mm":46.5,"culmen_depth_mm":14.5,"flipper_length_mm":213.0,"body_mass_g":4400.0,"groundtruth":"2"}
{"island":"Torgersen","culmen_length_mm":39.0,"culmen_depth_mm":17.1,"flipper_length_mm":191.0,"body_mass_g":3050.0,"groundtruth":"0"}

Baseline test:
{"island":"Dream","culmen_length_mm":36.4,"culmen_depth_mm":17.0,"flipper_length_mm":195.0,"body_mass_g":3325.0,"groundtruth":"0"}
{"island":"Torgersen","culmen_length_mm":36.7,"culmen_depth_mm":19.3,"flipper_leng

INFO:tensorflow:Assets written to: /tmp/tmpg6v7luu_/model/001/assets


Test accuracy: 0.6078431372549019


Let's test the inference code locally to ensure it works before deploying it. The `handler()` function is the entry point that will be called by SageMaker whenever the endpoint receives a request.

When testing the inference code, we want to set the `context` to `None` so the function recognizes we call it locally. We also want to set the `pipeline_s3_location` to the S3 location of the Scikit-Learn pipeline. We generated this pipeline in the preprocessing step from Session 1 and uploaded it to the location specified by the `proprocessor_destination` pipeline configuration variable.

In [34]:
from penguins.code.inference import handler

handler(
    data={
        "island": "Biscoe",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
    }, 
    context=None, 
    pipeline_s3_location=preprocessor_destination.default_value
)

Handling endpoint request
Processing input data...
Transforming input data...
Sending input data to model to make a prediction...
Response: {'predictions': [[0.2, 0.5, 0.3]]}
Processing prediction received from the model...
Prediction: 1. Confidence: 0.5


/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
 

('{"prediction": "1", "confidence": 0.5}', 'application/json')

## Pipeline Configuration

While building a pipeline, you only want to rerun every step if you expect a different result. To accomplish this, you can instruct SageMaker to reuse the result of a previous successful run of a pipeline step. You can find more information about this topic in [Caching Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html).

When creating a SageMaker Pipeline, we can specify a list of parameters we can use on individual pipeline steps. To read more about these parameters, check [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html).

These are the parameters we need:

* `dataset_location`: This parameter represents the dataset's location in S3. We will use this parameter to indicate the SageMaker Processing Job where to find the dataset. The Processing Job will download the dataset from S3 and make it available on the instance running the script.
* `preprocessor_destination`: We need to define the location where the SageMaker Processing Job will store the output. When it finishes, the Processing Job will copy the script's output to the S3 location specified by this parameter. By default, SageMaker uploads the output of a job to a custom location in S3, but unfortunately, if we rely on that functionality, we can't cache the Processing Step in the Pipeline.
* `train_dataset_baseline_destination`: This parameter represents the location where we will store the train dataset to compute constraints and statistic baselines to monitor the model.
* `test_dataset_baseline_destination`: This parameter represents the location where we will store the test dataset to compute constraints and statistic baselines to monitor the model.
* `timestamp_signature`: We'll use this parameter to automatically generate resources using a unique suffix to avoid collisions.
* `evaluation_destination`: This parameter represents the location where the Processing Step running the evaluation script will store the report. 
* `model_approval_status`: This parameter represents the default approval status we will use when registering a new model. Check [Update the Approval Status of a Model](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-approve.html) for more information about the different approval status of a model and how you can update them.
* `accuracy_threshold`: This parameter represents the minimum accuracy that the model should reach for it to be registered.

In [54]:
cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

dataset_location = ParameterString(
    name="dataset_location",
    default_value=INPUT_DATA_URI,
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f"{S3_FILEPATH}/preprocessing",
)

train_dataset_baseline_destination = ParameterString(
    name="train_dataset_baseline_destination",
    default_value=f"{S3_FILEPATH}/preprocessing/baselines/train",
)

test_dataset_baseline_destination = ParameterString(
    name="test_dataset_baseline_destination",
    default_value=f"{S3_FILEPATH}/preprocessing/baselines/test",
)

timestamp_signature = ParameterString(
    name="timestamp_signature",
    default_value="",
)

evaluation_destination = ParameterString(
    name="evaluation_destination",
    default_value=f'{S3_FILEPATH}/evaluation',
)

model_approval_status = ParameterString(
    name="model_approval_status", 
    default_value="Approved"
)

accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.70
)

## Pipeline Steps

### Preprocessing Step

The first step we need in the pipeline is a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) to run the preprocessing script. This Processing Step will create a SageMaker Processing Job in the background, run the script, and upload the output to S3. You can use Processing Jobs to perform data preprocessing, post-processing, feature engineering, data validation, and model evaluation. Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation for more information.

A processor gives the Processing Step information about the hardware and software that SageMaker should use to launch the Processing Job. To run the script, we need access to Scikit-Learn, so we can use the [SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) processor that comes out-of-the-box with the SageMaker's Python SDK. The [Data Processing with Framework Processors](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks.html) page discusses other built-in processors you can use. The [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) page contains information about the available framework versions for each region.

Here is the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep). Notice how it uses the processor that we defined above.

We also need to define the list of inputs that we need on the preprocessing script. In this case, the input is the dataset we stored in S3. 

We have a few outputs that we want SageMaker to capture when the Processing Job finishes. SageMaker will upload every one of these outputs to the location specified by the `preprocessor_destination` parameter except the baseline data, which we will upload to the location specified by the `baseline_destination` parameter.

In [13]:
sklearn_processor = SKLearnProcessor(
    base_job_name="penguins-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

preprocess_step = ProcessingStep(
    name="preprocessing",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=preprocessor_destination),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=preprocessor_destination),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=preprocessor_destination),
        ProcessingOutput(output_name="pipeline", source="/opt/ml/processing/pipeline", destination=preprocessor_destination),
        ProcessingOutput(output_name="train-baseline", source="/opt/ml/processing/train-baseline", destination=train_dataset_baseline_destination),
        ProcessingOutput(output_name="test-baseline", source="/opt/ml/processing/test-baseline", destination=test_dataset_baseline_destination),
    ],
    code=f"{PENGUINS_FOLDER}/preprocessor.py",
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


### Training Step

We can now create a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) that we can add to the pipeline. This Training Step will create a SageMaker Training Job in the background, run the training script, and upload the output to S3. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation for more information. 

SageMaker uses the concept of an [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to handle end-to-end training and deployment tasks. For this example, we will use the built-in [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) to run the training script we wrote before. The [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) page contains information about the available framework versions for each region. Here, you can also check the available SageMaker [Deep Learning Container images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md).

Notice the list of hyperparameters defined below. SageMaker will pass these hyperparameters as arguments to the entry point of the training script.

In [15]:
hyperparameters = {
    "epochs": 50,
    "batch_size": 32,
}

estimator = TensorFlow(
    entry_point=f"{PENGUINS_FOLDER}/train.py",
    hyperparameters=hyperparameters,
    framework_version="2.6",
    py_version="py38",
    instance_type="ml.m5.large",
    instance_count=1,
    script_mode=True,
    disable_profiler=True,
    role=role,
)

We can now create the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) using the estimator we defined before.

This step will receive the train and validation split from the preprocessing step as inputs. Notice how we reference both splits using the `preprocess_step` variable. This creates a dependency between the Training and Processing Step we defined in Session 1. When we build a new Pipeline, we'll see that the Training Step will run once the Processing Step finishes.

In [16]:
training_step = TrainingStep(
    name="training",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

### Evaluation Step

To run the evaluation script, we can use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing). Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation for more information.

Whenever you want to run a Processing Job using a machine learning framework, you can use an instance of the [FrameworkProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.FrameworkProcessor) class. For example, the [TensorFlowProcessor](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks-tensorflow.html) subclass will give you access to TensorFlow. You can also configure a Processing Job from scratch using a [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) instance combined with the [sagemaker.image_uris.retrieve()](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html) function for generating the URI of one of the  SageMaker pre-built docker images. 

This time, we will use a [TensorFlowProcessor](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job-frameworks-tensorflow.html) because we need our script to have access to TensorFlow and Scikit-Learn.

We can now create a [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) to run the evaluation script. 

The inputs of this step will be the model we created and the test set we generated during the preprocessing phase. The output will be the evaluation report file.

The [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) lets us specify a list of [PropertyFile](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.properties.PropertyFile) instances from the output of the job. We can use this to map the evaluation report generated in the evaluation script. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information.

In [17]:
tensorflow_processor = TensorFlowProcessor(
    framework_version="2.6",
    py_version="py38",
    base_job_name="penguins-evaluation-processor",
    instance_type="ml.m5.large",
    instance_count=1,
    role=role
)

# By default, the TensorFlowProcessor runs the script using
# /bin/bash as its entrypoint. We want to ensure we run it 
# using python3.
tensorflow_processor.framework_entrypoint_command = ["python3"]

# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
evaluation_report = PropertyFile(
    name="evaluation-report",
    output_name="evaluation",
    path="evaluation.json"
)


# Notice how this step uses the model generated by the tuning or training
# step, and the test set generated by the preprocessing step.
evaluation_step = ProcessingStep(
    name="evaluation",
    processor=tensorflow_processor,
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=evaluation_destination),
    ],
    code=f"{PENGUINS_FOLDER}/evaluation.py",
    property_files=[evaluation_report],
    cache_config=cache_config
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


### Create Model Step

To create the model quality baseline, we must generate predictions for the train set and compare them with the ground truth labels.

We can do this by running a Batch Transform Job to predict every sample from the training dataset. We can use a [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform) as part of the pipeline to run this job. You can check [Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) for more information about Batch Transform Jobs.

The Transform Step requires a model. In previous sessions, we only registered the model in the Model Registry as part of the pipeline, but now we also need to create the model. That way, the Batch Transform Job can use it to generate predictions.

Let's configure a [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-serving-model) and a Model Step to create the model.


In [36]:
model = TensorFlowModel(
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    entry_point="inference.py",
    source_dir=str(CODE_FOLDER),
    env={
        "PIPELINE_S3_LOCATION": preprocessor_destination,
    },
    framework_version="2.6",
    sagemaker_session=PipelineSession(),
    role=role,
)

create_model_step = ModelStep(
    name="create-model",
    step_args=model.create(instance_type="ml.m5.large"),
)

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


### Transform Step

Let's configure the [Batch Transform Job](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) using a [Transform Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform). This Batch Transform Job will run every sample from the training dataset through the model so we can compute the baseline metrics.

We can use an instance of the [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html) class to configure the job.





In [38]:
transformer = Transformer(
    # We can specify the name of the model the Batch Transform Job will 
    # use by using the property from the Model Step that created the model.
    model_name=create_model_step.properties.ModelName,
    instance_type="ml.c5.xlarge",
    instance_count=1,
    
    # The baseline set that we generated in the preprocessing step
    # is in JSON format, where every line is a JSON sample.
    accept="application/json",
    strategy="SingleRecord",
    assemble_with="Line",
    
    output_path=f"{S3_FILEPATH}/transform",
)

transform_step = TransformStep(
    name="transform",
    transformer=transformer,
    inputs=TransformInput(
        
        # We will use the test dataset we generated during the preprocessing 
        # step to run it through the model and generate predictions.
        data=preprocess_step.properties.ProcessingOutputConfig.Outputs["test-baseline"].S3Output.S3Uri,

        join_source="Input",
        content_type="application/json",
        split_type="Line",
    ),
    cache_config=cache_config
)

### Data Quality Check Step

Data quality monitoring allows us to monitor the quality of the data coming into our endpoint and detect whether it 
drifts away from the nature of the baseline data we used to train our model.

The first step is to create a baseline job using a [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) to analyze the train set. The baseline computes schema constraints and statistics for each feature in our data. Check the [QualityCheckStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.quality_check_step.QualityCheckStep) SageMaker's SDK documentation for more information.

Let's set up the configuration to monitor the quality of our data using Pipeline parameters. For more information about how to coordinate these parameters in different situations, check [ Baseline calculation, drift detection, and lifecycle with ClarifyCheck and QualityCheck steps in Amazon SageMaker Model Building Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-quality-clarify-baseline-lifecycle.html):

* `data_quality_skip_check`: Whether we should skip checking data drift during this pipeline execution. Set this to `True` if you are starting the pipeline for the first time to build your first model version or want to generate the initial baselines.
* `data_quality_register_new_baseline`: Whether you want to compute and register a new baseline.
* `data_quality_supplied_baseline_statistics`: You can use this property to specify the location of existing baseline statistics.
* `data_quality_supplied_baseline_constraints`: You can use this property to specify the location of existing baseline constraints.

Let's now configure the [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) and feed it the train set we generated in the preprocessing step.

We can configure the instance that will run the quality check using the [CheckJobConfig](https://sagemaker.readthedocs.io/en/v2.73.0/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.check_job_config.CheckJobConfig) class, and we can use the `DataQualityCheckConfig` class to configure the job.


In [39]:
data_quality_skip_check = ParameterBoolean(
    name="data_quality_skip_check", 
    default_value=True
)

data_quality_register_new_baseline = ParameterBoolean(
    name="data_quality_register_new_baseline", 
    default_value=True
)

data_quality_supplied_baseline_statistics = ParameterString(
    name="data_quality_supplied_baseline_statistics", 
    default_value=""
)

data_quality_supplied_baseline_constraints = ParameterString(
    name="data_quality_supplied_baseline_constraints", 
    default_value=""
)

In [40]:
check_job_config = CheckJobConfig(
    instance_type="ml.t3.xlarge",
    instance_count=1,
    sagemaker_session=sagemaker_session,
    volume_size_in_gb=20,
    role=role,
)

data_quality_check_config = DataQualityCheckConfig(
    # We will use the train dataset we generated during the preprocessing 
    # step to generate the data quality baseline.
    baseline_dataset=preprocess_step.properties.ProcessingOutputConfig.Outputs["train-baseline"].S3Output.S3Uri,
    
    dataset_format=DatasetFormat.json(lines=True),
    output_s3_uri=Join(on='/', values=[S3_FILEPATH, "monitoring", "data-quality"]),
)

data_quality_check_step = QualityCheckStep(
    name="data-quality-check",
    check_job_config=check_job_config,
    quality_check_config=data_quality_check_config,
    skip_check=data_quality_skip_check,
    register_new_baseline=data_quality_register_new_baseline,
    supplied_baseline_statistics=data_quality_supplied_baseline_statistics,
    supplied_baseline_constraints=data_quality_supplied_baseline_constraints,
    model_package_group_name=MODEL_PACKAGE_GROUP_NAME,
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### Model Quality Check Step

Model quality monitoring allows us to monitor the model's performance by comparing its predictions with the actual ground truth labels. 

The first step is to create a baseline job using a [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) to compare predictions from the model with the ground truth labels in the baseline dataset. The baseline job automatically creates baseline statistical rules and constraints that define thresholds against which we can evaluate the model performance. Check the [QualityCheckStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.quality_check_step.QualityCheckStep) SageMaker's SDK documentation for more information.

Let's set up the configuration to monitor the quality of our model using Pipeline parameters. For more information about how to coordinate these parameters in different situations, check [Baseline calculation, drift detection, and lifecycle with ClarifyCheck and QualityCheck steps in Amazon SageMaker Model Building Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-quality-clarify-baseline-lifecycle.html):

* `model_quality_skip_check`: Whether we should skip checking model drift during this pipeline execution. Set this to `True` if you are starting the pipeline for the first time to build your first model version or want to generate the initial baselines.
* `model_quality_register_new_baseline`: Whether you want to compute and register a new baseline.
* `model_quality_supplied_baseline_statistics`: You can use this property to specify the location of existing baseline statistics.
* `model_quality_supplied_baseline_constraints`: You can use this property to specify the location of existing baseline constraints.


Let's now configure the [Quality Check Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check) and feed it the data we generated in the Transform Step.

We will use the same [CheckJobConfig](https://sagemaker.readthedocs.io/en/v2.73.0/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.check_job_config.CheckJobConfig) we configured for the Data Quality Check Step.

In [41]:
model_quality_skip_check = ParameterBoolean(
    name="model_quality_skip_check", 
    default_value=True
)

model_quality_register_new_baseline = ParameterBoolean(
    name="model_quality_register_new_baseline", 
    default_value=True
)

model_quality_supplied_baseline_statistics = ParameterString(
    name="model_quality_supplied_baseline_statistics", 
    default_value=""
)

model_quality_supplied_baseline_constraints = ParameterString(
    name="model_quality_supplied_baseline_constraints", 
    default_value=""
)

In [42]:
model_quality_check_config = ModelQualityCheckConfig(
    # We are going to use the output of the Transform Step to generate
    # the model quality baseline.
    baseline_dataset=transform_step.properties.TransformOutput.S3OutputPath,
    
    dataset_format=DatasetFormat.json(lines=True),
    output_s3_uri=Join(on='/', values=[S3_FILEPATH, "monitoring", "model-quality"]),
    
    # We need to specify the problem type and the fields where the prediction
    # and groundtruth are so the process knows how to interpret the results.
    problem_type="MulticlassClassification",
    inference_attribute="$.SageMakerOutput.prediction",
    ground_truth_attribute="groundtruth",
)

model_quality_check_step = QualityCheckStep(
    name="model-quality-check",
    check_job_config=check_job_config,
    quality_check_config=model_quality_check_config,
    skip_check=model_quality_skip_check,
    register_new_baseline=model_quality_register_new_baseline,
    supplied_baseline_statistics=model_quality_supplied_baseline_statistics,
    supplied_baseline_constraints=model_quality_supplied_baseline_constraints,
    model_package_group_name=MODEL_PACKAGE_GROUP_NAME,
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### Register Model Step

In Session 4, we configured a set of [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) using the evaluation report we generated during the Evaluation Step. After running the Data and Model Quality Steps, we will have access to a much more comprehensive set of metrics that we can use when registering the model.

In [43]:
model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_check_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_check_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_check_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    
    model_constraints=MetricsSource(
        s3_uri=model_quality_check_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_check_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_check_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_check_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_check_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)


register_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.6",
        model_package_group_name=MODEL_PACKAGE_GROUP_NAME,
        model_metrics=model_metrics,
        drift_check_baselines=drift_check_baselines,
        approval_status=model_approval_status,
    )
)

### Deploy Step

In Session 4, we registered the model as part of the pipeline but deployed it manually. Instead, we can use a [Lambda Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda) to deploy the model automatically.

Let's start by writing the Lambda function to take the model information and create a new hosting endpoint.

Let's define the [LambdaStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.lambda_step.LambdaStep) that will run the function to deploy the model. In this example, this step will create a new Lambda function every time it runs, so we must remove any old function after the pipeline finishes.

We can use [Data Capture](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html) to record the inputs and outputs of the endpoint to use them later for monitoring the model. Amazon SageMaker Model Monitor automatically parses this captured data and compares metrics from this data with a baseline that you create for the model. 

We'll enable Data Capture and use the following settings:

* `data_capture_enabled`: Whether we want to enable Data Capture.
* `data_capture_percentage`: Represents the percentage of information that flows through the endpoint that we want to capture. For this example, we'll set that to 100%.
* `data_capture_destination`: Specifies the S3 location where we want to store the captured data.

In [46]:
%%writefile $PENGUINS_FOLDER/lambda.py

import os
import json
import boto3

sagemaker = boto3.client("sagemaker")

def lambda_handler(event, context):
    model_name = event["model_name"]
    endpoint_name = event["endpoint_name"]
    endpoint_config_name = event["endpoint_config_name"]
    data_capture_enabled = event["data_capture_enabled"]
    data_capture_percentage = event["data_capture_percentage"]
    data_capture_destination = event["data_capture_destination"]
    role = event["role"]
    
    
    # The first step is to create a new model. We will use the
    # ARN of the model we registered.
    sagemaker.create_model(
        ModelName=model_name, 
        ExecutionRoleArn=role, 
        Containers=[{
            "ModelPackageName": event["model_package_arn"]
        }] 
    )

    # Then, we need to create an Endpoint Configuration.
    # Here is where we specify the hardware we need for the
    # endpoint and the Data Capture configuration. 
    sagemaker.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "ModelName": model_name,
                "InstanceType": "ml.m5.large",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "VariantName": "AllTraffic",
            }
        ],
        DataCaptureConfig={
            "EnableCapture": data_capture_enabled,
            "InitialSamplingPercentage": data_capture_percentage,
            "DestinationS3Uri": data_capture_destination,
            "CaptureOptions": [
                {
                    'CaptureMode': "Input"
                },
                {
                    'CaptureMode': "Output"
                },
            ],
            "CaptureContentTypeHeader": {
                "JsonContentTypes": [
                    "application/json",
                    "application/octect-stream"
                ]
            }
        },
    )

    # Finally, we can create the new Endpoint using the
    # Endpoint Configuration we created above.
    sagemaker.create_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name,
    )
    
    return {
        "statusCode": 200,
        "body": json.dumps("Endpoint deployed successfully"),
        "model_name": model_name,
    }

Overwriting penguins/lambda.py


We need to ensure our Lambda function has permission to interact with SageMaker, so let's create a new role to run the function.

In [47]:
def create_lambda_role(role_name):
    try:
        response = iam_client.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description="Lambda Pipeline Role"
        )

        role_arn = response['Role']['Arn']

        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )

        iam_client.attach_role_policy(
            PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
            RoleName=role_name
        )

        return role_arn

    except iam_client.exceptions.EntityAlreadyExistsException:
        response = iam_client.get_role(RoleName=role_name)
        return response['Role']['Arn']
    

lambda_role = create_lambda_role("lambda-pipeline-role")

In [48]:
data_capture_enabled = ParameterBoolean(
    name="data_capture_enabled",
    default_value=True,
)

data_capture_percentage = ParameterInteger(
    name="data_capture_percentage",
    default_value=100,
)

data_capture_destination = ParameterString(
    name="data_capture_destination",
    default_value=f"{S3_FILEPATH}/monitoring/data-capture",
)


function_name = f"deploy-endpoint-fn-{time.strftime('%m%d%H%M%S', time.localtime())}"
endpoint_name = "penguins-endpoint"

deploy_step = LambdaStep(
    name="deploy",
    lambda_func=Lambda(
        function_name=function_name,
        execution_role_arn=lambda_role,
        script=str(PENGUINS_FOLDER / "lambda.py"),
        handler="lambda.lambda_handler",
        timeout=600,
        memory_size=10240,
    ),
    inputs={
        # We can use the timestamp_signature pipeline parameter
        # to add a suffix to the model and the endpoint configuration
        # and avoid name collisions.
        "model_name": Join(on="-", values=["penguins-model", timestamp_signature]),
        "endpoint_config_name": Join(on="-", values=["penguins-endpoint-config", timestamp_signature]),

        # We use the ARN of the model we registered to
        # deploy it to the endpoint.
        "model_package_arn": register_model_step.properties.ModelPackageArn,

        "endpoint_name": endpoint_name,
        "data_capture_enabled": data_capture_enabled,
        "data_capture_percentage": data_capture_percentage,
        "data_capture_destination": data_capture_destination,
        "role": role,
    },
    outputs=[
        LambdaOutput(output_name="model_name", output_type=LambdaOutputTypeEnum.String)
    ]
)


### Condition Step

We only want to compute the model quality baseline if the model's performance is above the predefined threshold. The [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) will gate all necessary steps to compute the baseline. 

In [50]:
condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ", 
        values=[
            "Execution failed because the model's accuracy was lower than", 
            accuracy_threshold
        ]
    ),
)

condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[
        create_model_step, 
        transform_step, 
        model_quality_check_step, 
        register_model_step,
        deploy_step
    ],
    else_steps=[fail_step], 
)

## Running the Pipeline

We can now run the pipeline. If the pipeline succeeds, there will be a new running endpoint.

In [51]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        train_dataset_baseline_destination,
        test_dataset_baseline_destination,
        timestamp_signature,
        data_capture_enabled,
        data_capture_percentage,
        data_capture_destination,
        data_quality_skip_check,
        data_quality_register_new_baseline,
        data_quality_supplied_baseline_statistics,
        data_quality_supplied_baseline_constraints,
        model_quality_skip_check,
        model_quality_register_new_baseline,
        model_quality_supplied_baseline_statistics,
        model_quality_supplied_baseline_constraints,
        evaluation_destination,
        model_approval_status,
        accuracy_threshold,
    ],
    steps=[
        preprocess_step, 
        data_quality_check_step,
        training_step, 
        evaluation_step,
        condition_step
    ],
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist or update it if it does.

We are going to overwrite the value of `timestamp_signature` to specify a unique time-based suffix that we'll use to generate the name of different resources we create during the pipeline.

In [52]:
pipeline.upsert(role_arn=role)
execution = pipeline.start(parameters=dict(
    timestamp_signature=time.strftime("%m%d%H%M%S", time.localtime())
))



INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


## Testing Endpoint

We can now create a [Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html) from the endpoint.

In [56]:
waiter = sagemaker_client.get_waiter("endpoint_in_service")
waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={
        "Delay": 10,
        "MaxAttempts": 30
    }
)

predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

In [57]:
predictor.predict({
    "island": "Dream",
    "culmen_length_mm": 46.4,
    "culmen_depth_mm": 18.6,
    "flipper_length_mm": 190.0,
    "body_mass_g": 3450.0,
    
})

{'prediction': '0', 'confidence': 0.662686348}

## Monitoring

### Generating Predictions and Ground Truth Data

Let's generate predictions and ground truth data to test the monitoring functionality.

We will repeatedly send every sample from the dataset to the endpoint and generate random ground truth data for the sake of this example.

In [171]:
# This event will let us stop the infinite threads
# that generate prediction and ground truth data.
stop_thread = Event()

ground_truth_path = f"{S3_FILEPATH}/monitoring/groundtruth"

# Let's use the original dataset to generate predictions and ground
# truth data. 
data = pd.read_csv(LOCAL_FILEPATH).dropna()

Let's generate prediction data using the original dataset.

In [172]:
def predict(predictor):
    for index in data.index:
        payload = {
            "island": data["island"][index],
            "culmen_length_mm": data["culmen_length_mm"][index],
            "culmen_depth_mm": data["culmen_depth_mm"][index],
            "flipper_length_mm": data["flipper_length_mm"][index],
            "body_mass_g": data["body_mass_g"][index],
        }

        predictor.predict(payload, inference_id=str(index))
        sleep(1)
        
        if stop_thread.is_set():
            break
    
def generate_prediction_data(predictor):
    while True:
        predict(predictor)
        if stop_thread.is_set():
            break
            

prediction_thread = Thread(
    target=generate_prediction_data,
    args=(predictor,)
)

Let's generate ground truth data and store it in S3. Notice the S3 path to save the ground truth data has the same path format as the data captured by the endpoint. Check [Ingest Ground Truth Labels and Merge Them With Predictions](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-merge.html) for more information about this.


In [173]:
def generate_ground_truth_record(inference_id):
    random.seed(inference_id)
    
    return {
        "groundTruthData": {
            "data": str(random.choice([0, 1, 2])),
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(records, upload_time):
    records = [json.dumps(r) for r in records]
    data = "\n".join(records)
    uri = f"{ground_truth_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    
    print(f"Uploading ground truth data to {uri}...")
    
    S3Uploader.upload_string_as_file_body(data, uri)

    
def generate_ground_truth_data(max_records):
    while True:
        records = [generate_ground_truth_record(i) for i in range(max_records)]
        upload_ground_truth(records, datetime.utcnow())
        
        if stop_thread.is_set():
            break
            
        sleep(30)
    
groundtruth_thread = Thread(
    target=generate_ground_truth_data,
    args=(len(data),)
)


Let's start both threads now. These threads will run forever until you restart the kernel or set the `stop_thread` event.

In [174]:
prediction_thread.start()
groundtruth_thread.start()

Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/1644.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/1714.jsonl...


### Checking the Captured Data

Let's check the S3 location where the endpoint stores the requests and responses that it receives. We defined this location using the `data_capture_destination` pipeline parameter.

Notice that it make take a few minutes for the first few files to show up in S3. Keep running the following line until you get some.

In [178]:
files = S3Downloader.list(data_capture_destination.default_value)
files[:5]

['s3://mlschool/penguins/monitoring/data-capture/penguins-endpoint/AllTraffic/2023/04/28/19/16-06-992-abf52eaa-40cd-4fb9-916a-96bfe20252c0.jsonl']

Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/1815.jsonl...


These files contain the data captured by the endpoint in a SageMaker-specific JSON-line format. Each inference request is captured in a single line in the `jsonl` file. The line contains both the input and output merged together.

Let's read the first line from the first file:

In [179]:
if len(files):
    lines = S3Downloader.read_file(files[0])
    print(json.dumps(json.loads(lines.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "application/json",
      "mode": "INPUT",
      "data": "{\"island\": \"Dream\", \"culmen_length_mm\": 46.4, \"culmen_depth_mm\": 18.6, \"flipper_length_mm\": 190.0, \"body_mass_g\": 3450.0}",
      "encoding": "JSON"
    },
    "endpointOutput": {
      "observedContentType": "application/json",
      "mode": "OUTPUT",
      "data": "{\"prediction\": \"0\", \"confidence\": 0.497521222}",
      "encoding": "JSON"
    }
  },
  "eventMetadata": {
    "eventId": "33532b83-7f7c-4335-9c8a-a8023c3799c6",
    "inferenceTime": "2023-04-28T19:16:06Z"
  },
  "eventVersion": "0"
}


### Monitoring Data Quality

We can now set up a schedule to continuously monitor data coming into the endpoint and compare it to the baseline we generated before. This monitoring job will use the baseline statistics and constraints we generated during the Data Quality Check Step. Check [Schedule Data Quality Monitoring Jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-schedule-data-monitor.html) for more information.

SageMaker looks for violations in the data captured by the endpoint. By default, they combine the input data with the endpoint output and compare the result with the previous baseline we generated. If we let SageMaker do this, we will get three violations:

1. An "extra column check" violation because the field `confidence` doesn't exist in the baseline.
2. An "extra column check" violation because the field `prediction` doesn't exist in the baseline.
3. A "missing column check" violation because the field `groundtruth` doesn't appear in the data captured from the endpoint.

We can fix these violations by creating a preprocessing script configuring the data we want the monitoring job to use. This script will create a `groundtruth` column, and exclude `confidence` and `prediction`. By doing this, we will not receive any of these three violations.


In [180]:
DATA_QUALITY_PREPROCESSOR = "data_quality_preprocessor.py"

Here is the preprocessing script for the Data Quality Monitoring Job. Check [Preprocessing and Postprocessing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-and-post-processing.html) for more information about how to configure these scripts.

In [200]:
%%writefile {PENGUINS_FOLDER}/{DATA_QUALITY_PREPROCESSOR}

def preprocess_handler(inference_record):
    input_data = inference_record.endpoint_input.data
    output_data = json.loads(inference_record.endpoint_output.data)
    
    response = json.loads(input_data)
    response["groundtruth"] = output_data["prediction"]
    return response

Overwriting penguins/data_quality_preprocessor.py


The monitoring schedule expects an S3 location pointing to the preprocessing script. Let's upload the script to the default bucket.

In [201]:
bucket = boto3.Session().resource("s3").Bucket(sagemaker_session.default_bucket())
prefix = "penguins-monitoring"
bucket.Object(os.path.join(prefix, DATA_QUALITY_PREPROCESSOR)).upload_file(str(PENGUINS_FOLDER / DATA_QUALITY_PREPROCESSOR))
data_quality_preprocessor = f"s3://{os.path.join(bucket.name, prefix, DATA_QUALITY_PREPROCESSOR)}"
data_quality_preprocessor

's3://sagemaker-us-east-1-325223348818/penguins-monitoring/data_quality_preprocessor.py'

Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/3921.jsonl...


We can now set up the Data Quality Monitoring Job using the [DefaultModelMonitor](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.DefaultModelMonitor) class. Notice how we specify the `record_preprocessor_script` using the S3 location where we uploaded our script.

In [202]:
data_monitor = DefaultModelMonitor(
    instance_type="ml.m5.xlarge",
    instance_count=1,
    max_runtime_in_seconds=3600,
    role=role,
)

data_monitor.create_monitoring_schedule(
    monitor_schedule_name="penguins-data-monitoring-schedule",
    endpoint_input=predictor.endpoint_name,
    record_preprocessor_script=data_quality_preprocessor,
    statistics=f"{S3_FILEPATH}/monitoring/data-quality/statistics.json",
    constraints=f"{S3_FILEPATH}/monitoring/data-quality/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: penguins-data-monitoring-schedule


Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/3951.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4021.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4051.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4121.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4151.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4222.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4252.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4322.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/20/4352.jsonl...
Uploading ground truth data to s3://mlschool/penguins/m

You can describe the schedule to see more information about the Data Quality Monitoring Job.

In [190]:
data_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:325223348818:monitoring-schedule/penguins-data-monitoring-schedule',
 'MonitoringScheduleName': 'penguins-data-monitoring-schedule',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2023, 4, 28, 19, 18, 30, 771000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 4, 28, 19, 18, 38, 360000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2023-04-28-19-18-29-791',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'penguins-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'penguins-data-monitoring-schedule',
  'ScheduledTime': datetime.datetime(2023, 4, 27, 19, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2023, 4, 27, 19, 6, 13, 320000, tzinfo=tzlocal()),
  'LastModifiedTime': datetime.datetime(2

Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/2850.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/2920.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/2950.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/3021.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/3051.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/3121.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/3151.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/3221.jsonl...
Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/19/3251.jsonl...
Uploading ground truth data to s3://mlschool/penguins/m

### Monitoring Model Quality

Let's set up a schedule to continuously monitor the quality of the model and compare it to the baseline we generated before. This monitoring job will use the baseline constraints we generated during the Model Quality Check Step. Check [Schedule Model Quality Monitoring Jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-schedule.html) for more information.

To set up a Model Quality Monitoring Job, we can use the [ModelQualityMonitor](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.ModelQualityMonitor) class. The [EndpointInput](https://sagemaker.readthedocs.io/en/v2.24.2/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.EndpointInput) instance configures the attribute the monitoring job should use to determine the prediction from the model.

Check [Amazon SageMaker Model Quality Monitor](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/model_quality/model_quality_churn_sdk.html) for a complete tutorial on how to run a Model Monitoring Job in SageMaker.

In [187]:
model_monitor = ModelQualityMonitor(
    instance_type="ml.m5.xlarge",
    instance_count=1,
    max_runtime_in_seconds=1800,
    role=role
)

endpoint_input = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    
    # The endpoint returns an attribute `prediction` with the
    # prediction from the model. That's the attribute we want to
    # use to compare with the ground truth.
    inference_attribute="prediction",
    
    destination="/opt/ml/processing/input_data",
)

model_monitor.create_monitoring_schedule(
    monitor_schedule_name="penguins-model-monitoring-schedule",
    endpoint_input=endpoint_input,
    problem_type="MulticlassClassification",
    ground_truth_input=ground_truth_path,
    output_s3_uri=f"{S3_FILEPATH}/monitoring/model-quality",
    constraints=f"{S3_FILEPATH}/monitoring/model-quality/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: penguins-model-monitoring-schedule


You can describe the schedule to see more information about the Model Quality Monitoring Job.

In [188]:
model_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:325223348818:monitoring-schedule/penguins-model-monitoring-schedule',
 'MonitoringScheduleName': 'penguins-model-monitoring-schedule',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2023, 4, 28, 19, 19, 8, 998000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 4, 28, 19, 19, 9, 91000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2023-04-28-19-19-08-331',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'penguins-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'penguins-model-monitoring-schedule',
  'ScheduledTime': datetime.datetime(2023, 4, 28, 15, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2023, 4, 28, 15, 7, 28, 127000, tzinfo=tzlocal()),
  'LastModifiedTime': datetime.datetime(

### Stopping Monitoring Jobs

Let's stop the monitoring jobs by deleting the monitoring schedules we created before.

In [204]:
data_monitor.delete_monitoring_schedule()
model_monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: penguins-data-monitoring-schedule


INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2023-04-28-20-39-24-206



Deleting Monitoring Schedule with name: penguins-model-monitoring-schedule


INFO:sagemaker.model_monitor.model_monitoring:Deleting Model Quality Job Definition with name: model-quality-job-definition-2023-04-28-19-19-08-331


We also need to stop the threads generating predictions and ground truth data.

In [205]:
stop_thread.set()

prediction_thread.join()
groundtruth_thread.join()

Uploading ground truth data to s3://mlschool/penguins/monitoring/groundtruth/2023/04/28/21/3044.jsonl...


## Cleaning up

Before you finish, don't forget to clean up after yourself.

In [58]:
predictor.delete_model()
predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: penguins-model-0502190721
INFO:sagemaker:Deleting endpoint configuration with name: penguins-endpoint-config-0502190721
INFO:sagemaker:Deleting endpoint with name: penguins-endpoint


In [59]:
pipeline.delete()

NameError: name 'session1_pipeline' is not defined

In [ ]:
# Let's delete every model we registered under our model package group
for mp in sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]:
    print(f"Deleting {mp['ModelPackageArn']}")
    sagemaker_client.delete_model_package(ModelPackageName=mp["ModelPackageArn"])

# We can now delete the model package group.    
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)